In [1]:

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf

/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df = pd.read_csv('/Users/harshitgupta/Desktop/vs /VS-Data-Den/rm_dataset.csv')


In [3]:
df = df.set_index('date_field')

In [4]:
df.columns

Index(['Unnamed: 0', 'regional_master', 'sales', 'flag', 'week_no'], dtype='object')

In [5]:
df.head()

,Unnamed: 0,regional_master,sales,flag,week_no
date_field,,,,,
2019-12-08,0,0.0,28994.45,active,49
2019-12-15,1,0.0,127699.87,active,50
2019-12-22,2,0.0,102609.00,active,51
2019-12-29,3,0.0,0.00,inactive,52
2020-01-05,4,0.0,0.00,inactive,1


In [6]:
df['flag'] = df['flag'].map({'inactive': 0, 'active': 1})

In [7]:
df_input=df[['regional_master','sales', 'flag', 'week_no']]

In [8]:
df_input

,regional_master,sales,flag,week_no
date_field,,,,
2019-12-08,0.0,28994.45,1,49
2019-12-15,0.0,127699.87,1,50
2019-12-22,0.0,102609.00,1,51
2019-12-29,0.0,0.00,0,52
2020-01-05,0.0,0.00,0,1
...,...,...,...,...
2024-02-04,50001297.0,16917.58,1,5
2024-02-11,50001297.0,12848.38,1,6
2024-02-18,50001297.0,20193.45,1,7


In [9]:
df_input.describe()

,regional_master,sales,flag,week_no
count,1.864400e+04,1.864400e+04,18644.000000,18644.000000
mean,4.939954e+07,2.547687e+05,0.887202,26.355289
std,5.447708e+06,1.014892e+06,0.316354,15.324225
min,0.000000e+00,-2.744000e+03,0.000000,1.000000
25%,5.000003e+07,3.969330e+03,1.000000,13.000000
50%,5.000007e+07,2.983912e+04,1.000000,26.000000
75%,5.000055e+07,1.581912e+05,1.000000,40.000000
max,5.000130e+07,2.177125e+07,1.000000,53.000000


In [10]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df_input)

In [11]:
features=data_scaled
target=data_scaled[:,0]

In [12]:
TimeseriesGenerator(features, target, length=2, sampling_rate=1, batch_size=1)[0]
     

(array([[[0.        , 0.00145763, 1.        , 0.92307692],
         [0.        , 0.00599081, 1.        , 0.94230769]]]),
 array([0.]))

In [13]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.20, shuffle = False)
     

In [14]:
x_train.shape

(14915, 4)

In [15]:
x_test.shape

(3729, 4)

In [16]:
win_length=720
batch_size=32
num_features=4
train_generator = TimeseriesGenerator(x_train, y_train, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator = TimeseriesGenerator(x_test, y_test, length=win_length, sampling_rate=1, batch_size=batch_size)
     

In [17]:
train_generator[0]

(array([[[0.00000000e+00, 1.45763082e-03, 1.00000000e+00, 9.23076923e-01],
         [0.00000000e+00, 5.99080943e-03, 1.00000000e+00, 9.42307692e-01],
         [0.00000000e+00, 4.83847762e-03, 1.00000000e+00, 9.61538462e-01],
         ...,
         [9.99974101e-01, 1.26021875e-04, 0.00000000e+00, 1.53846154e-01],
         [9.99974101e-01, 1.26021875e-04, 0.00000000e+00, 1.73076923e-01],
         [9.99974101e-01, 1.26021875e-04, 0.00000000e+00, 1.92307692e-01]],
 
        [[0.00000000e+00, 5.99080943e-03, 1.00000000e+00, 9.42307692e-01],
         [0.00000000e+00, 4.83847762e-03, 1.00000000e+00, 9.61538462e-01],
         [0.00000000e+00, 1.26021875e-04, 0.00000000e+00, 9.80769231e-01],
         ...,
         [9.99974101e-01, 1.26021875e-04, 0.00000000e+00, 1.73076923e-01],
         [9.99974101e-01, 1.26021875e-04, 0.00000000e+00, 1.92307692e-01],
         [9.99974101e-01, 1.26021875e-04, 0.00000000e+00, 2.11538462e-01]],
 
        [[0.00000000e+00, 4.83847762e-03, 1.00000000e+00, 9.615384

In [18]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape= (win_length, num_features), return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5)) 
model.add(tf.keras.layers.LSTM(128, return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5)) 
model.add(tf.keras.layers.Dropout(0.3)) 
model.add(tf.keras.layers.LSTM(64, return_sequences=False))
model.add(tf.keras.layers.Dropout(0.3)) 
model.add(tf.keras.layers.Dense(1))

2024-04-25 13:11:50.773208: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-04-25 13:11:50.773228: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-04-25 13:11:50.773233: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-04-25 13:11:50.773626: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-25 13:11:50.773640: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. W

In [19]:
import numpy as np

In [20]:
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [21]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode='min')
model.compile(loss=tf.losses.MeanSquaredError(), optimizer=tf.optimizers.Adam(), metrics=[tf.metrics.MeanAbsoluteError()])
    
history = model.fit(train_generator, epochs=10, validation_data=test_generator, callbacks=[early_stopping])

Epoch 1/10


2024-04-25 13:11:51.516188: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


444/444 ━━━━━━━━━━━━━━━━━━━━ 55s 121ms/step - loss: 0.0524 - mean_absolute_error: 0.1386 - val_loss: 4.0760e-05 - val_mean_absolute_error: 0.0035
Epoch 2/10
444/444 ━━━━━━━━━━━━━━━━━━━━ 53s 120ms/step - loss: 0.0071 - mean_absolute_error: 0.0676 - val_loss: 4.8114e-05 - val_mean_absolute_error: 0.0065
Epoch 3/10
444/444 ━━━━━━━━━━━━━━━━━━━━ 53s 120ms/step - loss: 0.0059 - mean_absolute_error: 0.0610 - val_loss: 2.6871e-05 - val_mean_absolute_error: 0.0050
Epoch 4/10
444/444 ━━━━━━━━━━━━━━━━━━━━ 54s 122ms/step - loss: 0.0050 - mean_absolute_error: 0.0561 - val_loss: 1.0386e-04 - val_mean_absolute_error: 0.0101
Epoch 5/10
444/444 ━━━━━━━━━━━━━━━━━━━━ 54s 122ms/step - loss: 0.0041 - mean_absolute_error: 0.0509 - val_loss: 2.6875e-04 - val_mean_absolute_error: 0.0164


In [22]:
model.evaluate(test_generator, verbose=0)  
     

[0.00026875073672272265, 0.016386089846491814]